# Federated Analysis - Genomics scenario

## Introduction

This notebook steps through an scenario of querying federated genomics data using the [Common API](https://github.com/federated-data-sharing/common-api). 
A task is defined to match SNPs of interest (based on genomic coordinate and mutation). 
Individual participant/patient/donor data is tested against a list of SNPs of interest. 
The results are then combined and visualised.

- This scenario was inspired by the datasets available at the [International Cancer Genome Consortium](https://dcc.icgc.org/). We acknowledge the generous data sharing policy and all the donors who participated
- In order to link SNPs to likely clinical consequences, we build on the work and data provided by [NIH ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/)

## Mutations of interest

The SNPs of interest are defined in a CSV file locally: [`top_mutations.csv`](./top_mutations.csv).

> In this version, the list is fixed and built into the containerised task.

In [ ]:
import pandas as pd
import os
import time
import threading

from automate import task_automate

In [ ]:
snps = pd.read_csv('top_mutations.csv')
snps.head()

## Build the container

The task is defined in the python script [`snp-match.py`](./snp-match.py) and a container is used to wrap up the task for execution at remote sites.
See also the shell script: [`build-docker-container.sh`](./build-docker-container.sh).

Skip this step if you only using a prebuilt container.

> It's generally a good idea to run the script locally first.

In [ ]:
!docker build . -t snp-match

In [ ]:
# Tag and push the image to the container registry
# Depneds on the ACR_REGISTRY environment variable
!sudo docker tag snp-match "$ACR_REGISTRY/snp-match:latest"
!sudo docker images | grep snp-match

In [ ]:
!sudo docker push "$ACR_REGISTRY/snp-match:latest"

## Executing remotely

In this scenario, the SNP data is distributed in N (N=3) sites. For simplicity, all nodes have the same API key (token) which is kept in the environment property `FDS_API_TOKEN`.

In [ ]:
if 'FDS_API_TOKEN' not in os.environ:
    print('Please ensure FDS_API_TOKEN is in the environment')
else:
    print('Found API token')

In [ ]:
# Note for now these are the same nodes...
# Depends on endpoint URLs e.g. node1 ... node3 at example.org
endpoints = [
     'https://node1.example.org/v1/api',
     'https://node2.example.org/v1/api',
     'https://node3.example.org/v1/api',
]

In [ ]:
# Define the task
task = {
    "task":{
        "name": "SNP Match",
        "description":"This task looks for SNPs of interest and returns unique donor counts per SNP.",
        "queryInput": {
            "selectionQuery": "{ snp_clean { donor_id chromosome chromosome_start chromosome_end mutated_from_allele mutated_to_allele } }"
        },
        "container": {
            "name":"snp-match",
            "tag":"latest",
            "registry":"covid19acregistry.azurecr.io"
        }
    }
}

In [ ]:
# TODO - move this to top
from zipfile import ZipFile

# A function to process zip file output from the task and return a dataframe
# This is specific to the task specified above
def process_zip(path_to_zipfile):
    print(f'Processing: {path_to_zipfile}')
    with ZipFile(path_to_zipfile) as zipfile:
        # TODO - make this more robust
        csv_file = zipfile.namelist()[0]
        with zipfile.open(csv_file) as csv:
            df = pd.read_csv(csv)
            return(df)

In [ ]:
# In this section, we run the task at each of the federated endpoints 
# using Python's concurrency support
from concurrent.futures import ThreadPoolExecutor

i = 0
with ThreadPoolExecutor(max_workers=len(endpoints)) as executor:
    futures = []
    for endpoint in endpoints:
        i = i + 1
        futures.append(executor.submit(task_automate, f'Task-{i}', endpoint, task, process_zip))
        time.sleep(1)

results = []
for f in futures:
    x = f.result()
    if x is not None:
        reference, df = x
        # Add a 'first' column
        df.insert(0, 'subgroup',reference)
        results.append(df)
    
print(len(results))


## Combine results from different pools of data

In [ ]:
df_final = pd.concat(results)
df_final

## Visualisation

In [ ]:
# TODO